In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
races_df = spark.read.parquet(f"{processed_folder_path}/races").withColumnRenamed("name", "race_name") \
                                                               .withColumnRenamed("race_timestamp", "race_date")

In [0]:
circuit_df = spark.read.parquet(f"{processed_folder_path}/circuits").withColumnRenamed("location", "circuit_location")

In [0]:
results_df = spark.read.parquet(f"{processed_folder_path}/results").withColumnRenamed("time", "race_time")

In [0]:
drivers_df = spark.read.parquet(f"{processed_folder_path}/drivers").withColumnRenamed("name", "driver_name") \
                                                                   .withColumnRenamed("number","driver_number")

In [0]:
constructors_df = spark.read.parquet(f"{processed_folder_path}/constructors").withColumnRenamed("nationality", "constructor_nationality").withColumnRenamed("name","team")

In [0]:
races_circuit_df = races_df.join(circuit_df, on="circuit_id", how="inner").select(
    races_df.race_id,
    races_df.race_name,
    races_df.race_year,
    races_df.race_date,
    circuit_df.circuit_location
)

In [0]:
race_results_df = results_df.join(races_circuit_df, on="race_id", how="inner") \
                            .join(drivers_df, on="driver_id", how="inner") \
                            .join(constructors_df, results_df['constructor_id'] == constructors_df['constructorId'], how="inner") \
                            .withColumn("created_date", current_timestamp())

In [0]:
final_df = race_results_df.select(
race_results_df.race_year,
race_results_df.race_name,
race_results_df.race_date,
race_results_df.circuit_location,
race_results_df.driver_name,
race_results_df.driver_number,
race_results_df.nationality,
race_results_df.team,
race_results_df.grid,
race_results_df.fastest_lap,
race_results_df.race_time,
race_results_df.points,
race_results_df.created_date,
race_results_df.position
)

In [0]:
display(final_df.filter("race_year == 2019 and circuit_location == 'Abu Dhabi'" ).orderBy(final_df.points.desc()))

In [0]:
final_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/race_results")